In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [7]:
processed_seoul = pd.read_csv('processed_seoul.csv')
processed_gyeonggi = pd.read_csv('processed_gyeonggi.csv')
processed_incheon = pd.read_csv('processed_incheon.csv')
processed_busan = pd.read_csv('processed_busan.csv')
processed_daegu = pd.read_csv('processed_daegu.csv')
processed_gwangju = pd.read_csv('processed_gwangju.csv')
processed_daejeon = pd.read_csv('processed_daejeon.csv')
processed_ulsan = pd.read_csv('processed_ulsan.csv')

In [8]:
# 도시별 df 딕셔너리
city_dfs = {
    '서울특별시': processed_seoul,
    '경기도': processed_gyeonggi,
    '인천광역시': processed_incheon,
    '부산광역시': processed_busan,
    '대구광역시': processed_daegu,
    '광주광역시': processed_gwangju,
    '대전광역시': processed_daejeon,
    '울산광역시': processed_ulsan
}

In [ ]:
"""
데이터셋 칼럼 설명

0   '권역': 행정 구역 기준 권역 정보 (수도권/지방)
1   '시군구': 시/군/구 단위 행정 구역명
2   '번지': 도로명 주소의 번지 정보
3   '본번': 도로명 주소의 본번
4   '부번': 도로명 주소의 부번
5   '단지명': 아파트 단지 이름
6   '계약년월': 계약 발생 연월 (YYYYMM)
7   '계약년도': 계약 발생 연도 (YYYY)
8   '계약월': 계약 발생 월 (1~12)
9   '계약일': 계약 발생 일 (1~31)
10  'contract_season': 계약 발생 계절 (봄/여름/가을/겨울)
11  'area_bin': 전용면적 기준 면적 구간
    - 분류 기준:
        - ≤60 ㎡ : 소형
        - 61~85 ㎡ : 중소형
        - 86~135 ㎡ : 중대형
        - >135 ㎡ : 대형
12  '전용면적(㎡)': 아파트의 전용면적 (㎡)
13  '거래금액(만원)': 실제 거래 금액 (만원)
14  'price_per_m2': 평당 가격 (만원/㎡)
15  '층': 거래된 주택의 층수
16  '건축년도': 건물 준공 연도
17  'building_age': 건물 연식 (계약년도 - 건축년도)
18  'is_new_building': 신축 여부 (True: 신축)
19  'log_거래금액': 거래금액(만원)의 로그 변환 값
20  'log_price_per_m2': price_per_m2의 로그 변환 값
"""

processed_seoul.head()

## 데이터 시각적 품질 진단
결과 -> task1_data_quality_viz_result.html

In [ ]:
numeric_vars = ['전용면적(㎡)', '거래금액(만원)', 'price_per_m2', '계약년도', '건축년도', 'building_age']
categorical_vars = ['area_bin', 'contract_season', 'is_new_building']

all_data = pd.DataFrame()

for city_name, df in city_dfs.items():
    print(f"=== {city_name.upper()} 주요 변수 분석 ===\n")
    
    # 1. 수치형 변수 히스토그램 + KDE
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    for i, var in enumerate(numeric_vars):
        ax = axes[i]
        bins = df[var].nunique() if var in ['계약년도', '건축년도'] else 30
        sns.histplot(df[var], bins=bins, kde=True, color='skyblue', edgecolor='black', ax=ax)
        ax.set_title(f'{var} 분포\n(평균: {df[var].mean():.2f}, 왜도: {df[var].skew():.2f})')
        ax.axvline(df[var].mean(), color='red', linestyle='--', alpha=0.8, label='평균')
        ax.legend(labels=['커널밀도', '평균'])
    plt.tight_layout()
    plt.show()
    
    # 2. 카테고리 변수 분포 (Matplotlib 사용)
    fig, axes = plt.subplots(1, len(categorical_vars), figsize=(18, 5))
    if len(categorical_vars) == 1:
        axes = [axes]
    for i, cat_var in enumerate(categorical_vars):
        counts = df[cat_var].value_counts()
        x = np.arange(len(counts))
        # 색상 리스트: 카테고리 개수에 맞춰 지정
        colors = plt.cm.tab10.colors[:len(counts)]
        axes[i].bar(x, counts.values, color=colors, edgecolor='black')
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(counts.index)
        axes[i].set_title(f'{cat_var} 분포')
        axes[i].set_xlabel(cat_var)
        axes[i].set_ylabel('빈도')
        for xi, val in zip(x, counts.values):
            axes[i].text(xi, val, str(val), ha='center', va='bottom', fontsize=9)
    plt.suptitle(f'{city_name.upper()} 카테고리 변수 분포', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    
    all_data = pd.concat([all_data, df], ignore_index=True)

# 3. 전체 도시 합친 데이터 시각화
print("=== 전체 도시 데이터 시각화 ===\n")

# 수치형 변수 히스토그램 + KDE
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()
for i, var in enumerate(numeric_vars):
    ax = axes[i]
    bins = all_data[var].nunique() if var in ['계약년도', '건축년도'] else 30
    sns.histplot(all_data[var], bins=bins, kde=True, color='lightgreen', edgecolor='black', ax=ax)
    ax.set_title(f'{var} 분포 (전체 도시)\n(평균: {all_data[var].mean():.2f}, 왜도: {all_data[var].skew():.2f})')
    ax.set_xlabel(var)
    ax.set_ylabel('빈도')
    ax.axvline(all_data[var].mean(), color='red', linestyle='--', alpha=0.8, label='평균')
    ax.legend(labels=['커널밀도', '평균'])
plt.tight_layout()
plt.show()

# 카테고리 변수 분포 (전체 도시)
fig, axes = plt.subplots(1, len(categorical_vars), figsize=(18, 5))
if len(categorical_vars) == 1:
    axes = [axes]
for i, cat_var in enumerate(categorical_vars):
    counts = all_data[cat_var].value_counts()
    x = np.arange(len(counts))
    colors = plt.cm.tab10.colors[:len(counts)]
    axes[i].bar(x, counts.values, color=colors, edgecolor='black')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(counts.index)
    axes[i].set_title(f'{cat_var} 분포 (전체 도시)')
    axes[i].set_xlabel(cat_var)
    axes[i].set_ylabel('빈도')
    for xi, val in zip(x, counts.values):
        axes[i].text(xi, val, str(val), ha='center', va='bottom', fontsize=9)
plt.suptitle('전체 도시 카테고리 변수 분포', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### 1) 데이터 품질 진단 결과 요약
- 거래금액과 평당 가격에서 강한 우측 치우침(극단치)이 관찰됨
- 면적 구간, 건물 연식, 신축 여부에 따라 뚜렷한 그룹별 분포 차이가 존재함

### 2) 관찰된 데이터 품질 문제
1. **가격 변수(거래금액 / price\_per\_m2)의 심한 왜곡 및 극단 거래 존재**
   * 거래금액과 평당 가격 분포가 심하게 왜곡되어 있으며, 초고가 거래가 포함돼 있음
   * 이로 인해 로그 변환한 변수(log_거래금액, log_price_per_m2)를 우선적으로 분석하는 것을 권장함
2. **이상치 및 입력 오류 의심 데이터 존재**
   * 전용면적 대비 비정상적으로 높은 평당 가격이 발견됨(초고가 의심)
   * 향후 임계값 설정 등을 통한 이상치 및 입력 오류 정밀 점검 필요
3. **범주형 변수 내 불균형 분포**
   * 면적 구간별 거래 건수가 특정 구간(예: 중소형 61~85㎡)에 편중돼 있을 가능성 존재
   * 권역별 거래량 불균형도 병행하므로, 지역 간 비교 시 거래량 가중치 적용 또는 충분한 표본이 확보된 시군구만 비교하는 기준 필터링이 필요함

### 3) 데이터 품질 문제가 분석에 미치는 영향
1. **평균 가격 지표의 왜곡 가능성**
   * 극단치들로 인해 산술 평균 가격이 부풀려져 잘못된 해석을 유발할 수 있음
   * 따라서 분석 시 중앙값 또는 로그 변환 평균(log-평균)을 사용하는 것이 더 신뢰성 높음
2. **지역 간 거래량 편중으로 인한 성장성 평가 왜곡**
   * 거래량이 상대적으로 적은 지역은 시장 성장성 판단에 오차가 커질 우려
   * 따라서 거래 건수 기반 가중치 부여 또는 ‘충분한 표본 수’ 기준을 둔 필터링이 반드시 병행돼야 함
3. **면적과 신축 여부 미통제 시 발생할 수 있는 통계적 교란(confounding)**
   * 수도권 내 신축 아파트나 중대형 면적 아파트 비중이 높을 경우, 단순 지역별 가격 비교가 교란 변수에 의해 왜곡될 수 있음
   * 분석 시 이러한 교란 요인들을 통제하거나 다변량 분석 기법 적용이 요구됨

## 수치형 변수 상관관계 분석
결과 -> task1_num_vars_corr_result.html

In [ ]:
"""
상호정보량(Mutual Information)과 상관계수 분석
- MI 계산: log_거래금액, log_price_per_m2에 대해 변수별 MI 계산, 0.1 이상 필터링
- 상관계수: Pearson/Spearman 상관계수 계산 및 히트맵 시각화
- 최종 결과: MI≥0.1이면서 |r|<0.2인 변수 출력
"""

from sklearn.feature_selection import mutual_info_regression

# 제외할 컬럼
exclude_cols = ['본번', '부번', '계약년월', '계약일', '거래금액(만원)', 'price_per_m2']

# 결과 저장용
final_results = { 'log_거래금액': {}, 'log_price_per_m2': {} }

# 함수: MI 계산 & 필터링
def compute_mi(X, y, threshold=0.1):
    mi = mutual_info_regression(X, y, random_state=42)
    mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
    return mi_series[mi_series >= threshold], mi_series

# 도시별 분석
for city, df in city_dfs.items():
    print(f"\n\n========== {city} ==========")
    
    numeric_df = df.select_dtypes(include=['int64', 'float64']).drop(columns=exclude_cols, errors='ignore').dropna()
    
    X = numeric_df.drop(columns=['log_거래금액', 'log_price_per_m2'], errors='ignore')
    y_targets = { 'log_거래금액': numeric_df['log_거래금액'], 'log_price_per_m2': numeric_df['log_price_per_m2'] }
    
    mi_results = {}
    for target_name, y in y_targets.items():
        filtered_mi, full_mi = compute_mi(X, y)
        mi_results[target_name] = full_mi
        print(f"\n[{target_name}] (MI>=0.1)")
        print(filtered_mi if not filtered_mi.empty else "조건 만족 없음")
    
    # MI 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20,6))
    for idx, (target_name, mi_series) in enumerate(mi_results.items()):
        sns.barplot(x=mi_series.index, y=mi_series.values, color='skyblue', ax=axes[idx])
        axes[idx].axhline(0.1, color='red', linestyle='--', linewidth=2, label='커트라인: 0.1')
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha='right')
        axes[idx].set_title(f"{city} - Mutual Information ({target_name})", fontsize=14)
        axes[idx].set_ylabel("MI 값")
        axes[idx].set_xlabel("변수")
        axes[idx].legend()
    plt.tight_layout()
    plt.show()
    
    # 상관계수
    pearson_corr = numeric_df.corr(method='pearson').loc[y_targets.keys(), :]
    spearman_corr = numeric_df.corr(method='spearman').loc[y_targets.keys(), :]
    
    fig, axes = plt.subplots(1,2, figsize=(20,8))
    sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=axes[0], cbar_kws={'label':'Pearson 상관계수'})
    axes[0].set_title(f'{city} - 피어슨 상관관계 (타겟 기준)', fontsize=14)
    
    sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=axes[1], cbar_kws={'label':'Spearman 상관계수'})
    axes[1].set_title(f'{city} - 스피어만 상관관계 (타겟 기준)', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    # MI >=0.1 & |r|<0.2 컬럼 후보
    for target_name, mi_series in mi_results.items():
        candidates = [col for col, mi_val in mi_series.items() if mi_val >= 0.1 
                      and col in pearson_corr.columns 
                      and (abs(pearson_corr.loc[target_name, col])<0.2 or abs(spearman_corr.loc[target_name, col])<0.2)]
        final_results[target_name][city] = candidates if candidates else "❌ 조건 만족 없음"

# 최종 결과 출력
print("\n\n========== 최종 결과: MI는 높지만 상관관계는 낮은 변수 ==========")
for target_name, city_dict in final_results.items():
    print(f"\n[{target_name}]")
    for city, result in city_dict.items():
        print(f"{city}: {result}")

### 카테고리형 변수 상관관계 분석
결과 -> task1_cat_vars_anova_result.html

In [ ]:
from scipy import stats
from statsmodels.stats.multitest import multipletests

def analyze_anova(
    city_dfs,
    categorical_cols=['area_bin', 'contract_season', 'is_new_building'],
    alpha=0.05,
    p_adjust_method='fdr_bh'  # 'bonferroni', 'fdr_bh', 'none'
):
    """
    도시별로 타겟(log_거래금액, log_price_per_m2)과 카테고리형 컬럼 간 ANOVA, Eta² 계산,
    다중비교 보정 적용, Boxplot 시각화, 요약 결과 DataFrame 반환.
    추가로 수도권(서울특별시, 경기도, 인천광역시)과 지방의 평균 Eta² 비교를 제공.

    Parameters
    ----------
    city_dfs : dict[str, pd.DataFrame]
        {도시명: 데이터프레임}
    categorical_cols : list[str]
        분석할 카테고리형 컬럼
    alpha : float
        유의수준
    p_adjust_method : str
        다중비교 보정법 ('bonferroni', 'fdr_bh', 'none')
    Returns
    -------
    results_df : pd.DataFrame
        열: [city, target, categorical_col, f_stat, p_value, p_adj, reject, eta_squared, eta_label, n_groups]
    """

    # 사용자 정의 순서(시각화용)
    area_bin_order = ['소형', '중소형', '중대형', '대형']
    contract_season_order = ['봄', '여름', '가을', '겨울']

    all_records = []
    targets = ['log_거래금액', 'log_price_per_m2']

    for city, df in city_dfs.items():
        print(f"\n\n========== {city} ANOVA 분석 ==========")

        # 결측 제거
        df = df.dropna(subset=targets + categorical_cols)

        for target in targets:
            print(f"\n--- 타겟: {target} ---")
            tmp_results = []  # 보정 전 임시 결과

            # 각 카테고리 변수별 ANOVA
            for cat_col in categorical_cols:
                # 그룹 분할
                groups = [df[target][df[cat_col] == g] for g in df[cat_col].unique()]
                groups = [g.dropna() for g in groups]
                n_groups = len(groups)

                if n_groups >= 2 and all(len(g) > 0 for g in groups):
                    # ANOVA
                    f_stat, p_value = stats.f_oneway(*groups)

                    # Eta Squared
                    grand_mean = df[target].mean()
                    ss_between = sum(len(g) * (g.mean() - grand_mean)**2 for g in groups)
                    ss_total = ((df[target] - grand_mean)**2).sum()
                    eta_squared = (ss_between / ss_total) if ss_total != 0 else 0.0

                    tmp_results.append({
                        'city': city,
                        'target': target,
                        'categorical_col': cat_col,
                        'f_stat': float(f_stat),
                        'p_value': float(p_value),
                        'eta_squared': float(eta_squared),
                        'n_groups': n_groups
                    })
                else:
                    print(f"{cat_col}: 그룹 수 부족 또는 비어 있는 그룹 존재로 ANOVA 생략")

            # 다중비교 보정
            if len(tmp_results) > 0:
                pvals = [r['p_value'] for r in tmp_results]
                if p_adjust_method == 'none':
                    p_adj = pvals
                    reject = [pv < alpha for pv in pvals]
                else:
                    reject, p_adj, _, _ = multipletests(pvals, alpha=alpha, method=p_adjust_method)

                # Eta² 해석 라벨
                def eta_label(v):
                    if v < 0.01: return '매우 작음'
                    if v < 0.06: return '작음'
                    if v < 0.14: return '중간'
                    return '큼'

                # 결과 병합 + 출력
                for r, adj, rej in zip(tmp_results, p_adj, reject):
                    r['p_adj'] = float(adj)
                    r['reject'] = bool(rej)
                    r['eta_label'] = eta_label(r['eta_squared'])
                    all_records.append(r)

                    print(f"{r['categorical_col']}: "
                          f"p = {r['p_value']:.4e}, "
                          f"p_adj({p_adjust_method}) = {r['p_adj']:.4e}, "
                          f"Eta² = {r['eta_squared']:.4f} [{r['eta_label']}]")

                # 유의 변수만 정렬 출력 (보정 p 기준)
                sig = [r for r in tmp_results if r['p_adj'] < alpha]
                sig.sort(key=lambda x: (x['p_adj'], -x['eta_squared']))

                if len(sig) > 0:
                    print(f"\n유의미한 컬럼 순서 (p_adj<{alpha}, {p_adjust_method})")
                    for r in sig:
                        print(f"- {r['categorical_col']}: "
                              f"p_adj={r['p_adj']:.4e}, Eta²={r['eta_squared']:.4f} [{r['eta_label']}]")
                else:
                    print(f"\n유의미한 컬럼 없음 (p_adj<{alpha}, {p_adjust_method})")

        # 시각화
        fig, axes = plt.subplots(len(categorical_cols), len(targets), figsize=(15, 10))
        for i, cat_col in enumerate(categorical_cols):
            for j, t in enumerate(targets):
                if cat_col == 'area_bin':
                    order = area_bin_order
                elif cat_col == 'contract_season':
                    order = contract_season_order
                else:
                    order = None
                sns.boxplot(x=cat_col, y=t, data=df, ax=axes[i, j], order=order)
                axes[i, j].set_title(f'{city} - {cat_col} vs {t}')
                axes[i, j].tick_params(axis='x', rotation=45)
        plt.tight_layout()
        plt.show()

    # 수도권 vs 지방 비교 분석
    results_df = pd.DataFrame(all_records, columns=[
        'city', 'target', 'categorical_col', 'f_stat', 'p_value', 'p_adj', 'reject', 'eta_squared', 'eta_label', 'n_groups'
    ])

    # 수도권 및 지방 분류
    capital_cities = ['서울특별시', '경기도', '인천광역시']
    results_df['region'] = results_df['city'].apply(lambda x: '수도권' if x in capital_cities else '지방')

    # 카테고리별 평균 Eta Squared 계산
    region_summary = results_df.groupby(['region', 'categorical_col'])['eta_squared'].mean().unstack()
    
    # 결과 출력
    print("\n=== 수도권 vs 지방: 평균 Eta Squared 비교 ===")
    print(region_summary)
    
    # 시각화: Barplot으로 비교
    region_summary.plot(kind='bar', figsize=(10, 6))
    plt.title('수도권 vs 지방: 카테고리별 평균 Eta Squared')
    plt.xlabel('지역')
    plt.ylabel('평균 Eta Squared')
    plt.xticks(rotation=0)
    plt.legend(title='카테고리')
    plt.tight_layout()
    plt.show()

    return results_df

analyze_anova(city_dfs)

### 1) 변수 상관관계 분석 요약
- **면적(전용면적 / area_bin)**: 가격을 설명하는 가장 중요한 변수로, 대부분 도시에서 압도적인 설명력을 보여줌
- **신축 여부(is_new_building)**: 평당가(log_price_per_m2)에서 지역별로 중~큰 효과를 나타내는 것으로 보임
- 계약계절(contract_season): 통계적으로 유의하지만, 가격 변동 설명력은 매우 작아 정책 및 투자 판단의 주요 변수는 아님

### 2) 수치형 변수
1. 전용면적(㎡)이 모든 도시에서 가장 높은 정보량(MI)을 보이며(즉 가장 강한 관련성), 그 다음으로 건축년도(또는 building_age) 순으로 영향을 줌
    - 여러 도시에서 log_거래금액·log_price_per_m2 기준 전용면적이 1위(값이 압도적)

2. 건축년도 / building_age는 일관되게 2·3위 수준의 설명력을 가짐
    - 연식(신축·노후성)이 가격·평당가 모두에 의미 있는 설명

3. 층, 계약년도 등은 일부 도시에서 MI가 조건부로 의미 있으나, 전용면적·연식 대비 영향력은 작음 


### 3) 카테고리 변수(ANOVA)
1. area_bin(면적구간): 거의 모든 광역시에서 **Eta²가 '큼'**으로 나타나 가격 변동의 주요 설명자 (예: 서울 0.3719, 경기도 0.3811, 부산 0.4536, 대전 0.5271)
    - 면적 분포 차이만으로도 지역별 가격 차의 상당 부분 설명됨

2. is_new_building(신축 여부): 일부 지방 도시에서 평당가에 큰 영향을 미치나, 수도권에서는 영향력이 상대적으로 작음
    - 인천 log_price_per_m2 Eta²=0.1604(큼), 대전 0.1116(중간), 부산 0.0973(중간) 등으로, 신축 비중이 높은 지역은 평당가 프리미엄이 강함
    - 반면 서울·경기도는 신축 효과가 상대적으로 작음(작음~중간)

3. contract_season(계약계절): p는 유의하지만 Eta²가 매우 작음(실무적 영향력은 미미)
    - 계절성은 있더라도 정책·투자 판단의 핵심 변수는 아님

### 4) 수도권 vs 지방 해석 포인트
수도권과 지방 간 가격 비교 시 면적 분포 차이 및 신축 여부에 따른 프리미엄 효과를 반드시 통제해야 한다.  
(지역 간 성장성 및 가격 차이가 오도될 가능성)